# Chapter 17: Advanced Patterns - Solutions
**From: Zero to AI Agent**

In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Exercise 17.1 Solution: Retry Logic

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
import random


class RetryState(TypedDict):
    input: str
    output: str
    error: str
    attempts: int
    max_attempts: int


def unreliable_operation(state: RetryState) -> dict:
    """Simulate an operation that sometimes fails."""
    attempts = state["attempts"] + 1
    
    # 40% chance of failure
    if random.random() < 0.4:
        print(f"Attempt {attempts}: Failed")
        return {"error": "Random failure", "attempts": attempts}
    
    print(f"Attempt {attempts}: Success!")
    return {
        "output": f"Processed: {state['input']}",
        "error": "",
        "attempts": attempts
    }


def should_retry(state: RetryState) -> str:
    if not state.get("error"):
        return "success"
    if state["attempts"] >= state["max_attempts"]:
        return "give_up"
    return "retry"


def handle_failure(state: RetryState) -> dict:
    return {"output": f"Failed after {state['attempts']} attempts"}


# Build graph
graph = StateGraph(RetryState)

graph.add_node("operation", unreliable_operation)
graph.add_node("handle_failure", handle_failure)

graph.set_entry_point("operation")
graph.add_conditional_edges(
    "operation",
    should_retry,
    {
        "success": END,
        "retry": "operation",
        "give_up": "handle_failure"
    }
)
graph.add_edge("handle_failure", END)

app = graph.compile()

# Test
result = app.invoke({
    "input": "test data",
    "output": "",
    "error": "",
    "attempts": 0,
    "max_attempts": 3
})

print(f"\nFinal result: {result['output']}")
print(f"Total attempts: {result['attempts']}")

---
## Congratulations!

You've completed Part 4: LangGraph!

Next: **Part 5: Production** (Chapter 18)